In [ ]:
file_list = glob.glob('data/raw_brain2d/*')


In [ ]:
for idx, h5file in enumerate(file_list):
    print(idx)
    with h5py.File(h5file, 'r') as hf:
        if 'sens' in hf.keys():
            sens_flag = True
        else:
            sens_flag = False
            kspace = hf['kspace'][:]
            kspace = kspace[:,:,::2]
    if sens_flag == False:
        num_slices = kspace.shape[0]
        sens = np.zeros(kspace.shape, dtype='complex64')

        for sl in range(num_slices):
            ksp = kspace[sl]    
            JsenseApp = mri.app.JsenseRecon(ksp, ksp_calib_width=20, show_pbar=False)
            sensemap = JsenseApp.run()
            del JsenseApp
            sensemap = sensemap.astype(np.complex64)
            sens[sl] = sensemap
        
        print('Writing a Sensitivity map for ', h5file)
        with h5py.File(h5file, 'a') as ha:
            del ha['kspace']
            ha.create_dataset('sens', data = sens)
            ha.create_dataset('kspace', data = kspace)
            print(kspace.shape, sens.shape) # Just in case

    else:
        print('Already got a sensitivity map')

In [3]:
from utils.dataset import Data2D

In [7]:
trainset = Data2D('data/div_brain2d/Test', dset_type='knee2d')
len(trainset)

0

In [ ]:
a,b,c,d = trainset[0]

In [ ]:
with h5py.File('data/raw_brain2d/file_brain_AXT2_203_2030225.h5', 'r') as hf:
    kspace = hf['kspace'][:]
print(kspace.shape)

In [ ]:
with h5py.File('data/div_brain2d/Train/file_brain_AXFLAIR_200_6002462.h5', 'a') as ha:
    del ha['kspace']
    ha.create_dataset('kspace', data = kspace)

In [ ]:
with h5py.File('data/div_brain2d/Train/file_brain_AXFLAIR_200_6002462.h5', 'r') as hf:
    kspace = hf['kspace'][:]
    print(kspace.shape)

In [ ]:
im = sp.ifft(kspace,axes=[2,3])

In [ ]:
im_comb = sp.rss(im,1)

In [ ]:
plt.imshow(np.abs(im_comb[8]), cmap='gray')